In [1]:
import os
os.chdir("../")
%pwd

'c:\\Codes\\Codes\\Python\\Summlytics'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainModelConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [3]:
from Summlytics.constants import *
from Summlytics.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_train_model_config(self) -> TrainModelConfig:

        config = self.config.train_model
        params = self.params.TrainingArgs

        create_directories([config.root_dir])

        return TrainModelConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

In [5]:
from Summlytics.logging import logger
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

c:\Codes\Codes\Python\Summlytics\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-02-09 01:52:01,982: INFO: config: PyTorch version 2.2.0+cu121 available.]


In [6]:
class TrainModel:
    def __init__(self, config: TrainModelConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        seq2seq_data_collator=DataCollatorForSeq2Seq(tokenizer, model=model)

        dataset = load_from_disk(self.config.data_path)

        training_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            warmup_steps=self.config.warmup_steps,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy,
            eval_steps=500,
            save_steps=1e6,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=dataset['test'],
            eval_dataset=dataset['validation'],
            tokenizer=tokenizer,
            data_collator=seq2seq_data_collator
        )

        trainer.train()
        model.save_pretrained(os.path.join(self.config.root_dir, 'pegasus-samsum-model'))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, 'pegasus-samsum-tokenizer'))

In [7]:
# try:
#     config = ConfigurationManager().get_train_model_config()
#     TrainModel(config).train()
# except Exception as e:
#     logger.error(e, exc_info=True)

try:
    config = ConfigurationManager()
    model_trainer_config = config.get_train_model_config()
    model_trainer_config = TrainModel(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-02-09 01:52:02,435: INFO: common: yaml file: <_io.TextIOWrapper name='config\\config.yaml' mode='r' encoding='cp1252'> loaded successfully.]
[2024-02-09 01:52:02,437: INFO: common: yaml file: <_io.TextIOWrapper name='params.yaml' mode='r' encoding='cp1252'> loaded successfully.]
[2024-02-09 01:52:02,437: INFO: common: Directory: artifacts already exists.]
[2024-02-09 01:52:02,438: INFO: common: Directory: artifacts/train_model already exists.]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
 20%|█▉        | 10/51 [02:40<12:09, 17.80s/it]

{'loss': 3.2001, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.2}


 39%|███▉      | 20/51 [05:18<08:12, 15.89s/it]

{'loss': 2.9017, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.39}


...

{'loss': 3.0946, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.78}


 98%|█████████▊| 50/51 [14:40<00:20, 20.12s/it]

{'loss': 2.9539, 'learning_rate': 5e-06, 'epoch': 0.98}


100%|██████████| 51/51 [14:58<00:00, 17.62s/it]
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


{'train_runtime': 898.6903, 'train_samples_per_second': 0.911, 'train_steps_per_second': 0.057, 'train_loss': 3.0422028092777027, 'epoch': 1.0}
